# TODO
1. Create Rules
2. Create ordering system

In [1]:
import copy
from numbers import Number

In [2]:
class Expression(): #Base Expression Class
    
    #Addition
    def __add__(self, other):
        return BinaryExpression(self, "+", other)
    def __radd__(self, other):
        return BinaryExpression(other, "+", self)
    
    #Subtraction
    def __sub__(self, other):
        return BinaryExpression(self, "-", other)
    def __rsub__(self, other):
        return BinaryExpression(other, "-", self)
    
    #Multiplication
    def __mul__(self, other):
        return BinaryExpression(self, "*", other)
    def __rmul__(self, other):
        return BinaryExpression(other, "*", self)
    
    #Division
    def __truediv__(self, other):
        return BinaryExpression(self, "/", other)
    def __rtruediv__(self, other):
        return BinaryExpression(other, "/", self)
    
    #Exponents
    def __pow__(self, other):
        return BinaryExpression(self, "^", other)
    def __rpow__(self, other):
        return BinaryExpression(other, "^", self)

In [48]:
class BinaryExpression(Expression): #Binary Expression Class
    left = None
    operator = None
    right = None
    
    def __init__(self, left, operator, right):
        self.left = left
        self.operator = operator
        self.right = right
        
    def __hash__(self):
        return hash(self.generateTreeString())
    
    def __eq__(self, other):
        return hash(self) == hash(other)
        
    def containsAdditionOrSubtraction(self): #Test if expression or children contain addition or subtraction
        if self.operator in ["+", "-"]: #Check self
            return True
        else:
            if isinstance(self.left, BinaryExpression): #Check left child
                leftContains = self.left.containsAdditionOrSubtraction()
            else:
                leftContains = False
            if isinstance(self.right, BinaryExpression): #Check right child
                rightContains = self.right.containsAdditionOrSubtraction()
            else:
                rightContains = False
            return leftContains or rightContains
        
    def generateTreeString(self):
        indentChar = "  "
        if isinstance(self.left, BinaryExpression):
            leftString = indentChar + self.left.generateTreeString().replace("\n", "\n" + indentChar)
        else:
            leftString = indentChar + str(self.left)
        if isinstance(self.right, BinaryExpression):
            rightString = indentChar + self.right.generateTreeString().replace("\n", "\n" + indentChar)
        else:
            rightString = indentChar + str(self.right)
        return self.operator + "\n" + leftString + "\n" + rightString
    
    def symbolSubstitution(self, substitutionDict): #NOT IN PLACE
        if not isinstance(substitutionDict, dict):
            raise TypeError("parameter substitutionDict must be a dictionary!")
        else:
            #Left
            newLeft = self.left
            if isinstance(self.left, BinaryExpression): #Recurse
                newLeft = self.left.symbolSubstitution(substitutionDict)
            elif isinstance(self.left, Symbol): #Substitute Symbol if in dict
                if self.left in substitutionDict.keys():
                    newLeft = substitutionDict[self.left]
            #Right
            newRight = self.right
            if isinstance(self.right, BinaryExpression): #Recurse
                newRight = self.right.symbolSubstitution(substitutionDict)
            elif isinstance(self.right, Symbol): #Substitute Symbol if in dict
                if self.right in substitutionDict.keys():
                    newRight = substitutionDict[self.right]
        return BinaryExpression(newLeft, copy.deepcopy(self.operator), newRight)
    
    def replacePattern(self, replace, capGroups=None, capModifiers=None): #NOT IN PLACE
        if capGroups is None: capGroups = {}
        if capModifiers is None: capModifiers = {}
        newExpr = copy.deepcopy(replace)
        substitutionDict = {}
        for groupNum in capGroups.keys():
            if groupNum in capModifiers.keys():
                substitutionDict[Symbol(str(groupNum))] = capModifiers[groupNum](capGroups[groupNum])
            else:
                substitutionDict[Symbol(str(groupNum))] = capGroups[groupNum]
        if len(substitutionDict.keys()) > 0:
            newExpr = newExpr.symbolSubstitution(substitutionDict)
        return newExpr

    def matchAndGather(self, match, capFilters=None): #Returns (ResultBool, CaptureGroups)
        if capFilters is None:
            capFilters = {}
        if hasattr(match, "__iter__"):
            for pattern in match:
                result = self.matchAndGather(match=pattern, capFilters=capFilters)
                if result[0]:
                    return result
            #None of the match patterns matched
            return (False, {})
        else:
            #Check if self matches
            if isinstance(self, match.__class__): #Self is correct class
                if self.operator == match.operator: #Self has correct operator
                    #Check children
                    selfChildren = (self.left, self.right)
                    matchChildren = (match.left, match.right)
                    results = []
                    for i in range(0, 2): #Iterate through left and right
                        if isinstance(matchChildren[i], Symbol): #Match child is Symbol
                            try: #The symbol name is a number, denoting a capture group
                                capGroupNum = int(matchChildren[i].name)
                                if capGroupNum in capFilters: #There is a filter
                                    if capFilters[capGroupNum](selfChildren[i]): #The filter evaluates True
                                        results.append((True, {capGroupNum: selfChildren[i]}))
                                    else: #The filter evaluates False
                                        results.append((False, {}))
                                else: #There is no filter
                                    results.append((True, {capGroupNum: selfChildren[i]}))
                            except ValueError: #The symbol name is not a number
                                if matchChildren[i] == selfChildren[i]: #The symbols match
                                    results.append((True, {}))
                                else: #The symbols do not match
                                    results.append((False, {}))
                        elif isinstance(matchChildren[i], BinaryExpression): #Match child is BinExp
                            if isinstance(selfChildren[i], BinaryExpression):
                                results.append(selfChildren[i].matchAndGather(matchChildren[i], capFilters))
                            else:
                                results.append((False, {}))
                        else:
                            raise ValueError("Unexpected class in match object: " + str(matchChildren[i].__class__))
                    if results[0][0] and results[1][0]: #Both children matched
                        totalResults = (True, {})
                        for r in results:
                            totalResults[1].update(r[1])
                        return totalResults
            elif isinstance(match, Symbol):
                try: #The symbol name is a number, denoting a capture group
                    capGroupNum = int(match.name)
                    if capGroupNum in capFilters: #There is a filter
                        if capFilters[capGroupNum](self): #The filter evaluates True
                            return(True, {capGroupNum: self})
                        else: #The filter evaluates False
                            return (False, {})
                    else: #There is no filter
                        return (True, {capGroupNum: self})
                except ValueError: #The symbol name is not a number
                    if match == self: #The symbols match
                        return (True, {})
                    else: #The symbols do not match
                        return (False, {})
            else:
                raise ValueError("unsupported type for match object")
            return (False, {})
    
    def matchAndReplace(self, match, replace, capFilters=None, capModifiers=None, preRecursion=False, postRecursion=False): #NOT IN PLACE
        newLeft = copy.deepcopy(self.left)
        newRight = copy.deepcopy(self.right)
        if preRecursion:
            if isinstance(self.left, BinaryExpression):
                newLeft = self.left.matchAndReplace(match,
                                                 replace,
                                                 capFilters=capFilters,
                                                 capModifiers=capModifiers,
                                                 preRecursion=True)
            if isinstance(self.right, BinaryExpression):
                newRight = self.right.matchAndReplace(match,
                                                  replace,
                                                  capFilters=capFilters,
                                                  capModifiers=capModifiers,
                                                  preRecursion=True)
        newBinExp = BinaryExpression(newLeft, copy.deepcopy(self.operator), newRight)
        matchResults = newBinExp.matchAndGather(match, capFilters)
        if matchResults[0]: #If self matched, replace
            newBinExp = newBinExp.replacePattern(replace, matchResults[1], capModifiers)
        if postRecursion and self != newBinExp and isinstance(newBinExp, BinaryExpression): #postRecursion on and this last pass changed something
            newBinExp = newBinExp.matchAndReplace(match,
                                                  replace,
                                                  capFilters,
                                                  capModifiers,
                                                  preRecursion,
                                                  postRecursion
                                                 )
        return newBinExp
    
    def applyRule(self, rule):
        newBinExp = self.matchAndReplace(rule.match,
                                    rule.replace,
                                    capFilters=rule.capFilters,
                                    capModifiers=rule.capModifiers,
                                    preRecursion=True,
                                    postRecursion=True
                                        )
        return newBinExp
    
    def matchRule(self, matchRule):
        return self.matchAndGather(matchRule.match, matchRule.capFilters)

In [49]:
class Rule():
    
    def __init__(self, match, replace, capFilters=None, capModifiers=None):
        self.match = match
        self.replace = replace
        self.capFilters=capFilters
        self.capModifiers=capModifiers

In [50]:
class MatchRule():
    
    def __init__(self, match, capFilters=None):
        self.match = match
        self.capFilters = capFilters

In [51]:
class Term(BinaryExpression): #Term Class (Binary Expression tree containing no + or - which can only be + or - with other expressions)
    
    def __init__(self):
        pass
    #Remove ability to multiply, divide, power?
    
    def reorderTerms(self):
        pass
    
    def expand(self):
        return self

In [52]:
class Symbol(Expression): #Symbol Class
    name = None
    
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name
    
    def __eq__(self, other):
        if isinstance(other, Symbol):
            if self.name == other.name:
                return True
        return False
    
    def __hash__(self):
        return hash(self.name)
    
    def symbolSubstitution(self, substitutionDict):
        if self in substitutionDict.keys():
            return copy.deepcopy(substitutionDict[self])
        
    def generateTreeString(self):
        return self.name
        
    def identityHash(self): #BAD! not necessarily identity because charcter codes have differing lengths
        if len(self.name) > 32:
            raise ValueError("name length > 32 not supported!")
        else:
            ords = [str(ord(c)) for c in self.name]
            hash_string = "".join(ords)
            hash_int = int(hash_string)
            return hash_int

In [53]:
class Rules():
    
    #Negative Exponents
    negativeExponentRule = Rule(
        match = Symbol("0") ** Symbol("1"),
        replace = 1 / (Symbol("0") ** Symbol("1")),
        capFilters = {1: lambda i: i<0 if isinstance(i, int) else False},
        capModifiers = {1: lambda i: i * -1}
    )
    
    #Identity Exponents
    identityExponentRule = Rule(
        match = Symbol("0") ** Symbol("1"),
        replace = Symbol("0"),
        capFilters = {1: lambda i: i == 1 if isinstance(i, int) else False}
    )
    
    #Exponent Power Rule
    powerRule = Rule(
        match = (Symbol("0") ** Symbol("1")) ** Symbol("2"),
        replace = Symbol("0") ** (Symbol("1") * Symbol("2"))
    )
    
    #Zero Exponents
    zeroExponentRule = Rule(
        match = Symbol("0") ** Symbol("1"),
        replace = Symbol("0"),
        capFilters = {1: lambda i: i == 0 if isinstance(i, int) else False},
        capModifiers = {0: lambda i: 1}
    )
    
    #Multiplicative Identity
    identityMultiplicationRule = Rule(
        match = [
            Symbol("0") * Symbol("1"),
            Symbol("1") * Symbol("0")
        ],
        replace = Symbol("0"),
        capFilters = {1: lambda i: i == 1 if isinstance(i, int) else False}
    )
    
    #No Subtraction (prefer addition)
    noSubtractionRule = Rule(
        match = Symbol("0") - Symbol("1"),
        replace = Symbol("0") + (-1 * Symbol("1"))
    )
    
    #Multiplication Expansion
    multiplicationExpandRule = Rule(
        match = [
            (Symbol("0") + Symbol("1")) * Symbol("2"),
            Symbol("2") * (Symbol("0") + Symbol("1"))
        ],
        replace = (Symbol("0") * Symbol("2")) + (Symbol("1") * Symbol("2"))
    )
    
    #Fix Exponents First
    exponentSimplifyRule = Rule(
        match = Symbol("0") ** Symbol("1"),
        replace = Symbol("0") ** Symbol("1"),
        capFilters = {1: lambda e: isinstance(e, BinaryExpression)},
        capModifiers= {1: lambda e: e.applyRule(Rules.multiplicationExpandRule)}
    )
    
    addEvaluableMatchRule = MatchRule(
        match = Symbol("0"),
#         capFilters = {0: lambda e: (e.operator == "+" \
#                                     and isinstance(e.left, Number) \
#                                     and (isinstance(e.right, Number) or e.right.matchRule(addEvaluableMatchRule)[0])) \
#                       if isinstance(e, BinaryExpression) else False}
        #capFilters = {0: lambda e: True if isinstance(e, BinaryExpression) else False}
    )

In [54]:
class Equality(): #Equality class
    pass

In [10]:
x = Symbol("x")
y = Symbol("y")
z = Symbol("z")
a = Symbol("a")
b = Symbol("b")

In [10]:
e = (x ** -1) ** -1
print(e.generateTreeString())

^
  ^
    x
    -1
  -1


In [11]:
e = e.applyRule(Rules.negativeExponentRule)
print("\n")
print(e.generateTreeString())



/
  1
  ^
    /
      1
      ^
        x
        1
    1


In [12]:
e = e.applyRule(Rules.identityExponentRule)
print(e.generateTreeString())

/
  1
  /
    1
    x


In [13]:
e = x ** 0

In [14]:
e = e.applyRule(Rules.zeroExponentRule)
print(e)

1


In [15]:
e = ((x + y) * (a + b)) * (Symbol("c") + Symbol("d"))
print(e.generateTreeString())

*
  *
    +
      x
      y
    +
      a
      b
  +
    c
    d


In [16]:
e = e.applyRule(Rules.multiplicationExpandRule)
print(e.generateTreeString())

+
  +
    +
      *
        c
        *
          a
          x
      *
        d
        *
          a
          x
    +
      *
        c
        *
          b
          x
      *
        d
        *
          b
          x
  +
    +
      *
        c
        *
          a
          y
      *
        d
        *
          a
          y
    +
      *
        c
        *
          b
          y
      *
        d
        *
          b
          y


In [18]:
e = a * (x + y + z)

In [20]:
e = e.applyRule(Rules.multiplicationExpandRule)
print(e.generateTreeString())

+
  +
    *
      x
      a
    *
      y
      a
  *
    z
    a


In [21]:
e = (x * (a + b)) * Symbol("c")

In [23]:
e = e.applyRule(Rules.multiplicationExpandRule)
print(e.generateTreeString())

+
  *
    *
      a
      x
    c
  *
    *
      b
      x
    c


In [24]:
e = (a + b) * (x + y)
e = e.applyRule(Rules.multiplicationExpandRule)
print(e.generateTreeString())

+
  +
    *
      x
      a
    *
      y
      a
  +
    *
      x
      b
    *
      y
      b


In [25]:
e = a * (x - y)

In [26]:
e = e.applyRule(Rules.noSubtractionRule)
e = e.applyRule(Rules.multiplicationExpandRule)
print(e.generateTreeString())

+
  *
    x
    a
  *
    *
      -1
      y
    a


In [27]:
e = x * 1

In [28]:
e = e.applyRule(Rules.identityMultiplicationRule)

In [29]:
print(e.generateTreeString())

x


In [40]:
e = ((x ** y) ** (a + b)) ** z

In [41]:
e = e.applyRule(Rules.powerRule)
e = e.applyRule(Rules.multiplicationExpandRule)

In [43]:
print(e.generateTreeString())

^
  x
  +
    *
      *
        a
        y
      z
    *
      *
        b
        y
      z


In [46]:
e = x ** (y * (a + b))

In [47]:
e = e.applyRule(Rules.exponentSimplifyRule)

In [48]:
print(e.generateTreeString())

^
  x
  +
    *
      a
      y
    *
      b
      y


In [55]:
e = BinaryExpression(5, "+", 6)

In [58]:
print(e.matchRule(Rules.addEvaluableMatchRule))

(True, {0: <__main__.BinaryExpression object at 0x7f10cc1f1908>})


In [57]:
e = BinaryExpression(5, "+", BinaryExpression(6, "+", Symbol("x")))

In [45]:
print(e.matchAndGather(Symbol("0")))

test
test0
(True, {0: <__main__.BinaryExpression object at 0x7f10cc5dc860>})
